In [4]:
import numpy as np
def bernoulli_samples(probability, size=1):
    """
    Generate samples from a Bernoulli distribution with a given probability.
    
    Args:
        probability (float): The probability of success (event occurring).
        size (int): The number of samples to draw. Default is 1.
        
    Returns:
        numpy.ndarray: An array of size `size` containing the generated samples.
    """
    return np.random.binomial(1, probability, size)

In [5]:

actual_frac_in_collision = 0.05
termination_frac_in_collision = 0.01 #0.01

#this is delta
delta = 0.01

tau = 0.5

def compute_sample_count(delta, tau, p):
    return 1/p*np.log(1/delta)*2/(tau**2)
# , p' = {actual_frac_in_collision:.3f}

#print(f"p'<= {(1-tau)*termination_frac_in_collision/(1+tau_minus) :.3f}")
print(f" Collision fraction threshold p = {termination_frac_in_collision:.3f}")
if actual_frac_in_collision>=termination_frac_in_collision:
    print(f"rejection probability expected more than if not true {1-delta:.3f}")
else:
    print(f"acceptance probability expected more than {1-delta:.3f} if p'<= {(1-tau)*termination_frac_in_collision/(1+tau_minus):.3f}")

prob_factor = 1/termination_frac_in_collision*np.log(1/delta)
tau_factors = [2/(tau**2)] #, (2+tau_minus)*(1+tau_minus)/(tau_minus**2*(1-tau))
required_sample_count = prob_factor*np.max(tau_factors)
required_sample_count = int(np.ceil(required_sample_count))
print(f"required sample count {required_sample_count}")
string =f"""required_sample_count type 2 err 
{prob_factor*tau_factors[0]}""" #, power {prob_factor*tau_factors[1]}
print(string)

 Collision fraction threshold p = 0.010
rejection probability expected more than if not true 0.990
required sample count 3685
required_sample_count type 2 err 
3684.1361487904733


In [6]:
delta = 0.01
p = 0.01
tau = 0.5

print(f'Want iris region that is {1-p} % collision free with {1-delta} certainty')

def compute_sample_count(delta, tau, p):
    return int(np.ceil(1/p*np.log(1/delta)*2/(tau**2)))

for i in range(1,10):
    delta_k = delta * np.pi**2/(6*(i**2))
    N = compute_sample_count(delta_k, tau, p)
    print(f"Iteration {i}: Required samples {N}, threshold to pass: lower than {N*(1-tau)*p:.2f} samples in collision")

Want iris region that is 0.99 % collision free with 0.99 certainty
Iteration 1: Required samples 3286, threshold to pass: lower than 16.43 samples in collision
Iteration 2: Required samples 4396, threshold to pass: lower than 21.98 samples in collision
Iteration 3: Required samples 5044, threshold to pass: lower than 25.22 samples in collision
Iteration 4: Required samples 5505, threshold to pass: lower than 27.53 samples in collision
Iteration 5: Required samples 5862, threshold to pass: lower than 29.31 samples in collision
Iteration 6: Required samples 6153, threshold to pass: lower than 30.77 samples in collision
Iteration 7: Required samples 6400, threshold to pass: lower than 32.00 samples in collision
Iteration 8: Required samples 6614, threshold to pass: lower than 33.07 samples in collision
Iteration 9: Required samples 6802, threshold to pass: lower than 34.01 samples in collision


In [26]:
delta_k = delta * np.pi**2/(6*(1000000**2))

In [27]:
compute_sample_count(delta_k, tau, p)

25391

In [7]:
from pydrake.all import HPolyhedron
import numpy as np

hp = HPolyhedron.MakeBox(-np.ones(5),  np.ones(5))


In [8]:
hp.MaximumVolumeInscribedEllipsoid().CalcVolume()

5.263788986222608

In [23]:
N_exp = 2000
print(f"running {N_exp} experiments")
res = []
means = []
for _ in range(N_exp):
    mean = np.mean(bernoulli_samples(actual_frac_in_collision, required_sample_count))
    result = 1.0 if mean<= (1-tau)*termination_frac_in_collision else 0
    res.append(result)
    means.append(mean)
print(f"est_accept_prob {np.mean(res):.3f}")
print(f"est_rej_prob {1-np.mean(res):.3f}")
print(f"{np.mean(means)}")
print(f"thresh {(1-tau)*termination_frac_in_collision}")

running 2000 experiments
est_accept_prob 0.000
est_rej_prob 1.000
0.049952781546811396
 thresh 0.005


0.051560379918588875

In [80]:
tau = 0.1
tau_minus = 0.3
print(f"decision thresold {1-tau}p")
print(f"accept power at p'<={(1-tau)/(1+tau_minus)}p")

decision thresold 0.9p
accept power at p'<=0.6923076923076923p


In [64]:
from scipy.stats import norm
import numpy as np

N = 1000
P0 = 0.90
alpha_z = 1e-20
mu = N*P0
sigma= np.sqrt(N*P0*(1-P0))
    
norm.ppf(alpha_z, loc = mu, scale= sigma)/N

0.8121297265594456

In [44]:
from scipy.stats import norm
import numpy as np

N = 1000
collision_free = 0.9
confidence = 1-1e-5
sigma = np.sqrt(N*collision_free*(1-collision_free))
norm.ppf(confidence, loc = N*collision_free, scale= sigma)


940.4603066420494

In [43]:
from scipy.stats import norm
import numpy as np

N = 100
collision_free = 0.99
confidence = 0.99
sigma = np.sqrt(N*collision_free*(1-collision_free))
res = norm.ppf(confidence, loc = N*collision_free, scale= sigma)
np.min([N, res])

100.0

In [10]:
from scipy.stats import norm, binom
import numpy as np

for confidence in [ 0.5, 0.8, 0.95, 0.99]:
    for N in [100, 500, 1000]:

        collision_free = 0.99
        sigma = np.sqrt(N*collision_free*(1-collision_free))
        res = int(norm.ppf(confidence, loc = N*collision_free, scale= sigma)+0.5)
        intres = np.min([res, N])
        max_achieveable_confidence = norm.cdf(N, loc = N*collision_free, scale= sigma)
        if intres ==N:
            print(f" Warning max achieveable confidence ~{max_achieveable_confidence:.3f}")
        print(f"N {N} confidence {confidence:.2f} collision_free {collision_free:.2f} #colfree particles {intres}")

N 100 confidence 0.50 collision_free 0.99 #colfree particles 99
N 500 confidence 0.50 collision_free 0.99 #colfree particles 495
N 1000 confidence 0.50 collision_free 0.99 #colfree particles 990
 Warning max achieveable confidence ~0.843
N 100 confidence 0.80 collision_free 0.99 #colfree particles 100
N 500 confidence 0.80 collision_free 0.99 #colfree particles 497
N 1000 confidence 0.80 collision_free 0.99 #colfree particles 993
 Warning max achieveable confidence ~0.843
N 100 confidence 0.95 collision_free 0.99 #colfree particles 100
N 500 confidence 0.95 collision_free 0.99 #colfree particles 499
N 1000 confidence 0.95 collision_free 0.99 #colfree particles 995
 Warning max achieveable confidence ~0.843
N 100 confidence 0.99 collision_free 0.99 #colfree particles 100
 Warning max achieveable confidence ~0.988
N 500 confidence 0.99 collision_free 0.99 #colfree particles 500
N 1000 confidence 0.99 collision_free 0.99 #colfree particles 997


In [44]:
sigma

0.9949874371066204

In [7]:
norm.ppf(0.9, loc = 0, scale= 1)

1.2815515655446004

In [40]:

graph = np.array([[0, 1, 1, 1, 1],
                  [0, 0, 1, 1, 1],
                  [0, 0, 0, 1, 1],
                  [0, 0, 0, 0, 1],
                  [0, 0, 0, 0, 0],
                  ])

triplets = [[0,1,1],
            [0,2,1],
            [1,2,1],
            [3,6,1],
            [3,7,1],
            [3,8,1],
            [4,6,1],
            [4,7,1],
            [4,8,1],
            [5,6,1],
            [5,7,1],
            [5,8,1],
            ]

# graph = np.zeros((9,9))
# for t in triplets:
#     graph[t[0], t[1]] = t[2]

import numpy as np
import networkx as nx
np.random.seed(0)
for _ in range(100):
    n = 1000
    graph = np.zeros((n,n))
    for i in range(n):
        for j in range(i+1, n):
            if np.random.rand()>0.1:
                graph[i,j] = 1

    graph = graph + graph.T

    is_clique_member = np.zeros((graph.shape[0],))

    def compute_degrees(graph):
        degrees = []
        for i in range(graph.shape[0]):
            degrees.append(np.sum(graph[i, :]))
        return degrees

    def pick_best(degrees_sort_idx, available_nodes):
        first_seen = False
        for d in degrees_sort_idx:
            if d in available_nodes:
                first_seen = True
            if d in available_nodes and first_seen:
                return d

    def update_available_nodes(adj_mat, point_to_add, available_nodes):
        update = []
        for a in available_nodes:
            if adj_mat[a, point_to_add]:
                if not a==point_to_add:
                    update.append(a)
        return update

    def remove_non_neighbour_edges(curr_graph, point_to_add):
        for i in range(curr_graph.shape[0]):
            if not curr_graph[i, point_to_add]:
                for j in range(curr_graph.shape[0]):
                    curr_graph[i,j] = 0
        return curr_graph
    
    available_nodes = np.arange(graph.shape[0])
    clique_members = []

    curr_graph = graph.copy()
    degrees_original = compute_degrees(curr_graph)
    degrees_original_argsort = np.argsort(degrees_original)[::-1] 
    while True:
        degrees = compute_degrees(curr_graph)
        #degrees_sort_idx = 
        point_to_add = np.argsort(degrees)[::-1][1] #pick_best(degrees_sort_idx, available_nodes)
        alt_sol = pick_best(degrees_original_argsort, available_nodes)
        print(f"once sort res {alt_sol} alg pick {point_to_add}")
        print(f"degrees_original oncesort {degrees_original[alt_sol]} alg {degrees_original[point_to_add]}")
        if(degrees_original[alt_sol] != degrees_original[point_to_add]):
            print('ERRRROOOOOOORR##########')
            break
        clique_members.append(point_to_add)
        available_nodes = update_available_nodes(curr_graph, point_to_add, available_nodes)
        if len(available_nodes)== 0:
            break
        curr_graph = remove_non_neighbour_edges(curr_graph, point_to_add)

    #fill in result
    for i in clique_members:
        is_clique_member[i] = 1 
    g = nx.Graph(graph)

#nx.draw(g, node_color = ['r' if i else 'k' for i in is_clique_member])


once sort res 794 alg pick 274
degrees_original oncesort 924.0 alg 922.0
ERRRROOOOOOORR##########
once sort res 294 alg pick 623
degrees_original oncesort 930.0 alg 926.0
ERRRROOOOOOORR##########
once sort res 963 alg pick 768
degrees_original oncesort 926.0 alg 925.0
ERRRROOOOOOORR##########
once sort res 223 alg pick 92
degrees_original oncesort 929.0 alg 926.0
ERRRROOOOOOORR##########
once sort res 54 alg pick 776
degrees_original oncesort 926.0 alg 926.0
once sort res 54 alg pick 366
degrees_original oncesort 926.0 alg 921.0
ERRRROOOOOOORR##########
once sort res 971 alg pick 355
degrees_original oncesort 928.0 alg 928.0
once sort res 7 alg pick 736
degrees_original oncesort 926.0 alg 924.0
ERRRROOOOOOORR##########
once sort res 978 alg pick 983
degrees_original oncesort 929.0 alg 925.0
ERRRROOOOOOORR##########
once sort res 987 alg pick 851
degrees_original oncesort 934.0 alg 931.0
ERRRROOOOOOORR##########
once sort res 757 alg pick 714
degrees_original oncesort 927.0 alg 925.0
ER

KeyboardInterrupt: 

In [30]:
from pydrake.all import IrisOptions

In [31]:
io = IrisOptions()
print(io)

IrisOptions(require_sample_point_is_contained=False, iteration_limit=100, termination_threshold=0.02, relative_termination_threshold=0.001, configuration_space_margin=0.01, num_collision_infeasible_samples=5, configuration_obstacles [], prog_with_additional_constraints is not set, num_additional_constraint_infeasible_samples=5, random_seed=1234, mixing_steps=10)


In [32]:
from iris_environments.environments import env_names
import yaml
default = {
    'require_sample_point_is_contained': True,
    'iteration_limit': 100,
    'termination_threshold': 0.02,
    'relative_termination_threshold': 0.001,
    'configuration_space_margin': 0.01,
    'num_collision_infeasible_samples': 5,
    'num_additional_constraint_infeasible_samples': 5,
    'random_seed': 1234
}

for e in env_names:
    with open(f"../benchmarks/default_options/{e}_12312354.yml", "w") as f:
        yaml.dump(default, f)

FileNotFoundError: [Errno 2] No such file or directory: '../benchmarks/default_options/2DOFFLIPPER_12312354.yml'

In [10]:
options = IrisOptions()

